<a href="https://colab.research.google.com/github/bu11ymaguire/studying/blob/main/ReinforcementLearning/montecarlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ReinforcementLearning/

#Monte_Carlo - Sample_Model

In [1]:
import numpy as np

def sample(dices = 2):
  x = 0
  for _ in range(dices):
    x += np.random.choice([1,2,3,4,5,6])
  return x

In [2]:
for _ in range(3):
  print(sample())

7
8
8


분포 모델의 경우 일일히 확률 분포를 계산해야되기 때문에 경우의 수가 많아질수록 매우 귀찮아진다. 이에, 샘플링을 많이해서 평균을 구하는 방법이 도입되었고, 이 방법이 몬테카를로 방법이다.

#Implement monte_carlo

In [3]:
trial = 1000

samples = []

for _ in range(trial):
  s = sample()
  samples.append(s)
  V = sum(samples) / len(samples)
  print(V)

7.0
8.0
7.0
6.75
6.6
7.0
6.714285714285714
7.25
7.333333333333333
7.5
7.090909090909091
7.25
7.615384615384615
7.571428571428571
7.666666666666667
7.3125
7.352941176470588
7.222222222222222
7.368421052631579
7.3
7.190476190476191
7.363636363636363
7.434782608695652
7.458333333333333
7.64
7.6923076923076925
7.592592592592593
7.678571428571429
7.586206896551724
7.6
7.67741935483871
7.78125
7.7272727272727275
7.852941176470588
7.9714285714285715
8.0
8.0
7.973684210526316
7.923076923076923
7.975
8.048780487804878
7.928571428571429
7.953488372093023
7.909090909090909
7.977777777777778
7.891304347826087
7.851063829787234
7.770833333333333
7.6938775510204085
7.62
7.588235294117647
7.653846153846154
7.69811320754717
7.62962962962963
7.618181818181818
7.625
7.684210526315789
7.724137931034483
7.762711864406779
7.666666666666667
7.704918032786885
7.709677419354839
7.634920634920635
7.5625
7.538461538461538
7.5606060606060606
7.582089552238806
7.5588235294117645
7.594202898550725
7.64285714285714

In [4]:
#증분구현

trial = 1000
V, n = 0, 0

for _ in range(trial):
  s = sample()
  n += 1
  V += (s - V) / n
  print(V)

8.0
6.5
5.333333333333333
6.25
6.0
6.0
5.857142857142857
6.125
6.666666666666667
6.800000000000001
7.090909090909092
7.083333333333334
7.000000000000001
7.214285714285715
7.200000000000001
7.187500000000001
7.352941176470589
7.444444444444445
7.315789473684211
7.05
7.0
6.954545454545454
7.173913043478261
7.041666666666667
7.04
7.153846153846154
7.2592592592592595
7.214285714285714
7.206896551724138
7.1
7.161290322580645
7.1875
7.181818181818182
7.205882352941177
7.228571428571429
7.194444444444445
7.216216216216217
7.236842105263158
7.256410256410257
7.300000000000001
7.414634146341464
7.476190476190476
7.488372093023256
7.5
7.466666666666667
7.478260869565218
7.382978723404256
7.395833333333334
7.387755102040817
7.420000000000001
7.372549019607844
7.403846153846154
7.377358490566038
7.407407407407407
7.454545454545454
7.410714285714286
7.350877192982456
7.431034482758621
7.3559322033898304
7.4
7.409836065573771
7.435483870967742
7.444444444444445
7.421875
7.369230769230769
7.424242424